In [3]:
from langchain_community.document_loaders import HuggingFaceDatasetLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.indexes.vectorstore import VectorstoreIndexCreator

In [4]:
dataset_name = "tweet_eval"
page_content_column = "text"
name = "stance_climate"

In [5]:
loader = HuggingFaceDatasetLoader(dataset_name, page_content_column, name)

In [6]:
embeddings = HuggingFaceEmbeddings()

index = VectorstoreIndexCreator(embedding = embeddings).from_loaders([loader])

/tmp/ipykernel_145639/1640482908.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embeddings = HuggingFaceEmbeddings()
/tmp/ipykernel_145639/1640482908.py:1: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings()
/home/jbaudry/Documents/DATABIRD/RAG_DATABIRD/rag388_env/lib/python3.8/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/us

In [7]:
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline

hf = HuggingFacePipeline.from_model_id(
    model_id = "openai-community/gpt2",
    task = "text-generation",
    device = -1,
    pipeline_kwargs = {"max_new_tokens": 50},
    model_kwargs = {"temperature": 0}
)

/home/jbaudry/Documents/DATABIRD/RAG_DATABIRD/rag388_env/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/jbaudry/Documents/DATABIRD/RAG_DATABIRD/rag388_env/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Device has 1 GPUs available. Provide device={deviceId} to `from_model_id` to use availableGPUs for execution. deviceId is -1 (default) for

In [9]:
def generate_answer(question):
    result = index.query(question, llm = hf)
    target_string = "Helpful Answer:"
    start_index = result.find(target_string)
    if start_index != -1:
        return result[start_index + len(target_string):].strip()
    else:
        return "No helpful answer found."



In [10]:
print(generate_answer("What is climate change ?"))

We now find out. Climate change is nothing more than a product of a few events and the "accidental" effect of the fossil fuels that have been used as a tool. It takes place in many, many places. So we can get an


In [2]:
import gradio as gr



In [3]:
demo = gr.Interface(fn = generate_answer, 
                    inputs = gr.Textbox(lines = 2, placeholder = "Enter a question"), 
                    outputs = "text", 
                    title = "Climate Change RAG", 
                    description = "Ask anything about climate change")

demo.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "/home/jbaudry/Documents/DATABIRD/RAG_DATABIRD/rag388_env/lib/python3.8/site-packages/gradio/queueing.py", line 536, in process_events
    response = await route_utils.call_process_api(
  File "/home/jbaudry/Documents/DATABIRD/RAG_DATABIRD/rag388_env/lib/python3.8/site-packages/gradio/route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
  File "/home/jbaudry/Documents/DATABIRD/RAG_DATABIRD/rag388_env/lib/python3.8/site-packages/gradio/blocks.py", line 1935, in process_api
    result = await self.call_function(
  File "/home/jbaudry/Documents/DATABIRD/RAG_DATABIRD/rag388_env/lib/python3.8/site-packages/gradio/blocks.py", line 1520, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
  File "/home/jbaudry/Documents/DATABIRD/RAG_DATABIRD/rag388_env/lib/python3.8/site-packages/anyio/to_thread.py", line 56, in run_sync
    return await get_async_backend().run_sync_in_worker_thr